In [2]:
!python pca_pipeline.py

In [3]:
import pandas as pd

pca_df = pd.read_csv('temp/pca_df.csv')
normalized_input = pd.read_csv('temp/normalized_input.csv')
grid_id = 
# X_train = normalized_input.iloc[:,2:-1]
# y_train = normalized_input.iloc[:,-1]

In [4]:
from itertools import product
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import pickle as pk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [5]:
# PCA to n = 2

features = normalized_input.columns[2:] #input.columns[1:]  # Exclude the first column, which is the index

# Select the number of components (in this case, reduce to 2 dimensions)
n_components = 2

# Instantiate the PCA model
# pca = PCA(n_components=n_components)

pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=n_components))])

# Fit and transform the data
pca_result = pipeline.fit_transform(normalized_input[features])

# Create a DataFrame with the PCA results
pca_df = pd.DataFrame(data=pca_result, columns=[f'PC{i+1}' for i in range(n_components)])

# Concatenate the first column (index) from pca_input to pca_df
pca_df['domain'] = normalized_input['domain']

In [ ]:
print(grid_id)

In [ ]:
pca_df

In [ ]:
pca_df['label'] = 0

pca_df.loc[pca_df.domain.isin(cb),'label'] = 1
pca_df['label'].sum()

In [ ]:
# Plotting pca_df with ground truth labels
pca_df1 = pca_df
plt.figure(figsize=(10, 6))
plt.scatter(pca_df1['PC1'], pca_df1['PC2'], c=pca_df1['label'], cmap='viridis', alpha=0.75)
plt.title(f'{grid_id}_PCA scatterplot all-all')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.colorbar(label='Index')


plt.savefig(f"{SAVE_DIR}/{grid_id}_PCA-all-all.jpg")

In [ ]:
# Zoomed plot of pca_df with ground truth labels

pca_df1 = pca_df[pca_df.PC1<20]
plt.figure(figsize=(10, 6))
plt.scatter(pca_df1['PC1'], pca_df1['PC2'], c=pca_df1['label'], cmap='viridis', alpha=0.75)
plt.title(f'{grid_id}_PCA scatterplot all-all zoomed')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.colorbar(label='Index')

# plt.show()

plt.savefig(f"{SAVE_DIR}/{grid_id}_PCA-all-all-zoomed.jpg")

In [ ]:
pca_df

In [ ]:
# kNN train-test split
kmeans = KMeans(n_clusters=2, random_state=0)
predicted_labels = kmeans.fit_predict(pca_df[['PC1', 'PC2']])
pca_df['predicted'] = predicted_labels
print(grid_id)


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(pca_df['label'], pca_df['predicted'])

In [ ]:
from sklearn.metrics import recall_score, precision_score, accuracy_score
recall = recall_score(pca_df['label'], pca_df['predicted'])
acc = accuracy_score(pca_df['label'], pca_df['predicted'])
precision = precision_score(pca_df['label'], pca_df['predicted'])
print(acc)
print(precision)
print(recall)


In [ ]:
# Plotting train_result_df with ground truth labels
plt.figure(figsize=(10, 6))
plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['predicted'], cmap='viridis', alpha=0.75)
plt.title(f'PCA Scatterplot train cross on {grid_id}train')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.colorbar(label='Index')


plt.savefig(f'pca-feb-20/KNN_cross_test_on_{grid_id}_all.jpg')

# 3. Building PCA with train-test split on 333519

In [ ]:
print(grid_id)

In [ ]:
n_components = 2
pca_train = PCA(n_components=n_components)

# Fit and transform the data
train_result = scale.fit_transform(X_train)
train_result = pca_train.fit_transform(train_result)

# Create a DataFrame with the PCA results
train_result_df = pd.DataFrame(data=train_result, columns=[f'PC{i+1}' for i in range(n_components)])

train_result_df.set_index(X_train.index)

# train_result_df['label'] = y_train
train_result_df = train_result_df.assign(label=y_train.values)


In [ ]:
# Plotting train_result_df with ground truth labels
plt.figure(figsize=(10, 6))
plt.scatter(train_result_df['PC1'], train_result_df['PC2'], c=train_result_df['label'], cmap='viridis', alpha=0.75)
plt.title(f'{grid_id}_PCA scatterplot train-train')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.colorbar(label='Index')


plt.savefig(f"{SAVE_DIR}/{grid_id}_PCA-train-train.jpg")

In [ ]:
# Zoomed plot of pca_df with ground truth labels

train_result_df1 = train_result_df[train_result_df.PC1<20]
train_result_df1 = train_result_df1[train_result_df1.PC2<20]
# Plotting train_result_df with ground truth labels
plt.figure(figsize=(10, 6))
plt.scatter(train_result_df1['PC1'], train_result_df1['PC2'], c=train_result_df1['label'], cmap='viridis', alpha=0.75)
plt.title(f'{grid_id}_PCA scatterplot train-train zoomed')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.colorbar(label='Index')


plt.savefig(f"{SAVE_DIR}/{grid_id}_PCA-train-train zoomed.jpg")

In [ ]:
test_result = scale.fit_transform(X_test)

test_result = pca_train.transform(test_result)

test_result_df = pd.DataFrame(data=test_result, columns=[f'PC{i+1}' for i in range(n_components)])

test_result_df.set_index(X_test.index)

# train_result_df['label'] = y_train
test_result_df = test_result_df.assign(label=y_test.values)


In [ ]:
# Plotting train_result_df with ground truth labels
plt.figure(figsize=(10, 6))
plt.scatter(test_result_df['PC1'], test_result_df['PC2'], c=test_result_df['label'], cmap='viridis', alpha=0.75)
plt.title(f'{grid_id}_PCA scatterplot train-test')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.colorbar(label='Index')

plt.savefig(f"{SAVE_DIR}/{grid_id}_PCA-train-test zoomed.jpg")

# 3b. Saving the trained PCA model

In [ ]:
pk.dump(pca_train, open(f"pca_{grid_id}.pkl","wb"))

In [ ]:
# pca_reloaded = pk.load(open("pca_333519.pkl",'rb')) 
# result_new = pca_reloaded.transform(X_test) # X_new is a new data sample

In [ ]:
test_result = pca_train.transform(X_test)

test_result_df = pd.DataFrame(data=test_result, columns=[f'PC{i+1}' for i in range(n_components)])

test_result_df.set_index(X_test.index)

# train_result_df['label'] = y_train
test_result_df = test_result_df.assign(label=y_test.values)

In [ ]:
# # Plotting train_result_df with ground truth labels
# plt.figure(figsize=(10, 6))
# plt.scatter(test_result_df['PC1'], test_result_df['PC2'], c=test_result_df['label'], cmap='viridis', alpha=0.75)
# plt.title(f'{grid_id}_PCA scatterplot train-test')
# plt.xlabel('Principal Component 1 (PC1)')
# plt.ylabel('Principal Component 2 (PC2)')
# plt.colorbar(label='Index')


# plt.savefig(f"{SAVE_DIR}/{grid_id}_PCA-train-test.jpg")

In [ ]:
# kNN train-test split
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit_predict(train_result_df[['PC1', 'PC2']])
predicted_labels = kmeans.predict(test_result_df[['PC1', 'PC2']])
test_result_df['predicted'] = predicted_labels
print(grid_id)


In [ ]:
test_result_df

In [ ]:
grid_id

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_result_df['label'], test_result_df['predicted'])

In [ ]:
from sklearn.metrics import recall_score, precision_score, accuracy_score
recall = recall_score(test_result_df['label'], test_result_df['predicted'])
acc = accuracy_score(test_result_df['label'], test_result_df['predicted'])
precision = precision_score(test_result_df['label'], test_result_df['predicted'])
print(acc)
print(precision)
print(recall)


# 4. Testing 333519's model on 333346

In [ ]:
grid_id

In [ ]:
pca_reloaded = pk.load(open("pca_333519.pkl",'rb')) 
result_new = pca_reloaded.transform(X_test) # X_new is a new data sample

In [ ]:

# Fit and transform the data
train_result = scale.fit_transform(X_train)
train_result = pca_reloaded.fit_transform(train_result)

# Create a DataFrame with the PCA results
train_result_df = pd.DataFrame(data=train_result, columns=[f'PC{i+1}' for i in range(2)])

train_result_df.set_index(X_train.index)

# train_result_df['label'] = y_train
train_result_df = train_result_df.assign(label=y_train.values)


In [ ]:
# Plotting train_result_df with ground truth labels
plt.figure(figsize=(10, 6))
plt.scatter(train_result_df['PC1'], train_result_df['PC2'], c=train_result_df['label'], cmap='viridis', alpha=0.75)
plt.title(f'PCA Scatterplot train cross on {grid_id}train')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.colorbar(label='Index')


plt.savefig(f'pca-feb-20/PCA_cross_test_on_{grid_id}_all.jpg')

In [ ]:
grid_id

In [ ]:
# Zoomed plot of pca_df with ground truth labels

train_result_df1 = train_result_df[train_result_df.PC1<20]
train_result_df1 = train_result_df1[train_result_df1.PC2<20]
# Plotting train_result_df with ground truth labels
plt.figure(figsize=(10, 6))
plt.scatter(train_result_df1['PC1'], train_result_df1['PC2'], c=train_result_df1['label'], cmap='viridis', alpha=0.75)
plt.title('PCA Scatterplot train')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.colorbar(label='Index')


plt.savefig(f'pca-feb-20/PCA_cross_test_on_{grid_id}.jpg')

In [ ]:
# kNN train-test split
kmeans = KMeans(n_clusters=2, random_state=0)
predicted_labels = kmeans.fit_predict(train_result_df[['PC1', 'PC2']])
train_result_df['predicted'] = predicted_labels
print(grid_id)


In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(train_result_df['label'], train_result_df['predicted'])

In [ ]:
from sklearn.metrics import recall_score, precision_score, accuracy_score
recall = recall_score(train_result_df['label'], train_result_df['predicted'])
acc = accuracy_score(train_result_df['label'], train_result_df['predicted'])
precision = precision_score(train_result_df['label'], train_result_df['predicted'])
print(acc)
print(precision)
print(recall)


In [ ]:
# Plotting train_result_df with ground truth labels
plt.figure(figsize=(10, 6))
plt.scatter(train_result_df['PC1'], train_result_df['PC2'], c=train_result_df['predicted'], cmap='viridis', alpha=0.75)
plt.title(f'PCA Scatterplot train cross on {grid_id}train')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.colorbar(label='Index')


plt.savefig(f'pca-feb-20/KNN_cross_test_on_{grid_id}_all.jpg')

In [ ]:
# Zoomed plot of pca_df with ground truth labels

train_result_df1 = train_result_df[train_result_df.PC1<20]
train_result_df1 = train_result_df1[train_result_df1.PC2<20]
# Plotting train_result_df with ground truth labels
plt.figure(figsize=(10, 6))
plt.scatter(train_result_df1['PC1'], train_result_df1['PC2'], c=train_result_df1['predicted'], cmap='viridis', alpha=0.75)
plt.title('PCA Scatterplot train')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.colorbar(label='Index')


plt.savefig(f'pca-feb-20/KNN_cross_test_on_{grid_id}.jpg')

# 5. Testing PCA